### 0. 환경설정
`requests` 설치

In [ ]:
! pip install requests --upgrade

### 1. 데이터 입수

In [ ]:
import requests
url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/CAN.csv'
response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    print('First 100 characters of data are')
    print(response.text[:100])

### 2. CSV 데이터 처리

In [ ]:
with open('test01.csv', 'r') as reader:
    for line in reader:
        print(len(line))

In [ ]:
with open('test02.csv', 'r') as reader:
    for line in reader:
        fields = line.split(',')
        print(fields)

In [ ]:
import csv

with open('test02.csv', 'r') as raw:
    cooked = csv.reader(raw)
    for record in cooked:
        print(record)

In [ ]:
import csv

with open('test02.csv', 'r') as raw:
    lines = raw.readlines()
cooked = csv.reader(lines)
for record in cooked:
    print(record)

In [ ]:
with open('test01.csv', 'r') as reader:
    data = reader.read()
    lines = data.split('\n')
    print(lines)

In [ ]:
with open('test01.csv', 'r') as reader:
    data = reader.read()
    lines = data.strip().split('\n')
    print(lines)

In [ ]:
import requests
import csv

url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/CAN.csv'
response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    wrapper = csv.reader(response.text.strip().split('\n'))
    for record in wrapper:
        print(record)

In [ ]:
import requests
import csv

url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/CAN.csv'
response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    wrapper = csv.reader(response.text.strip().split('\n'))
    for record in wrapper:
        if record[0] != 'year':
            year = int(record[0])
            value = float(record[1])
            print(year, value)

### 3. 오류처리와 일반화

In [ ]:
def annual_mean_temp(country):
    ''' ("CAN" 처럼) 3-문자로 된 ISO코드로 특정 국가에 대한 연평균 기온정보를 얻어온다.'''
    url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/' + country + '.csv'
    response = requests.get(url)
    if response.status_code != 200:
        print('Failed to get data:', response.status_code)
    else:
        wrapper = csv.reader(response.text.strip().split('\n'))
        results = []
        for record in wrapper:
            if record[0] != 'year':
                year = int(record[0])
                value = float(record[1])
                results.append([year, value])
        return results

In [ ]:
canada = annual_mean_temp('CAN')
print('first three entries for Canada:', canada[:3])

In [ ]:
latveria = annual_mean_temp('LTV')
print 'first three entries for Latveria:', latveria[:3]

In [ ]:
def annual_mean_temp(country):
    ''' ("CAN" 처럼) 3-문자로 된 ISO코드로 특정 국가에 대한 연평균 기온정보를 얻어온다.'''
    url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/' + country + '.csv'
    print('url used is', url)
    response = requests.get(url)
    print('response code:', response.status_code)
    print('length of data:', len(response.text))
    if response.status_code != 200:
        print('Failed to get data:', response.status_code)
    else:
        wrapper = csv.reader(response.text.strip().split('\n'))
        results = []
        for record in wrapper:
            if record[0] != 'year':
                year = int(record[0])
                value = float(record[1])
                results.append([year, value])
        return results

latveria = annual_mean_temp('LTV')
print('number of records for Latveria:', len(latveria))

In [ ]:
def annual_mean_temp(country):
    ''' 
    ("CAN" 처럼) 3-문자로 된 ISO코드로 특정 국가에 대한 연평균 기온정보를 얻어온다.
    만약 국가코드가 적법하지 않다면, 빈 리스트를 반환하라.
    '''
    url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/' + country + '.csv'
    response = requests.get(url)
    results = []
    if len(response.text) > 0:
        wrapper = csv.reader(response.text.strip().split('\n'))
        for record in wrapper:
            if record[0] != 'year':
                year = int(record[0])
                value = float(record[1])
                results.append([year, value])
    return results

print('number of records for Canada:', len(annual_mean_temp('CAN')))
print('number of records for Latveria:', len(annual_mean_temp('LTV')))

In [ ]:
def diff_records(left, right):
    '''[year, value] 리스트 짝이 주어지면, [year, difference] 리스트 짝을 반환하라.'''
    num_years = len(left)
    results = []
    for i in range(num_years):
        left_year, left_value = left[i]
        right_year, right_value = right[i]
        difference = left_value - right_value
        results.append([left_year, difference])
    return results

In [ ]:
print('one record:', diff_records([[1900, 1.0]],
                                  [[1900, 2.0]]))
print('two records:', diff_records([[1900, 1.0], [1901, 10.0]],
                                   [[1900, 2.0], [1901, 20.0]]))

In [ ]:
print('mis-matched years:', diff_records([[1900, 1.0]],
                                         [[1999, 2.0]]))
print('left is shorter', diff_records([[1900, 1.0]],
                                      [[1900, 10.0], [1901, 20.0]]))
print('right is shorter', diff_records([[1900, 1.0], [1901, 2.0]],
                                       [[1900, 10.0]]))

In [ ]:
def diff_records(left, right):
    '''
    [year, value] 리스트 짝이 주어지면, [year, difference] 리스트 짝을 반환하라.
    만약 입력이 정확하게 대응되는 년도가 아니라면 동작하지 않는다.
    '''
    assert len(left) == len(right), \
           'Inputs have different lengths.'
    num_years = len(left)
    results = []
    for i in range(num_years):
        left_year, left_value = left[i]
        right_year, right_value = right[i]
        assert left_year == right_year, \
               'Record {0} is for different years: {1} vs {2}'.format(i, left_year, right_year)
        difference = left_value - right_value
        results.append([left_year, difference])
    return results

In [ ]:
print('one record:', diff_records([[1900, 1.0]],
                                  [[1900, 2.0]]))
print('two records:', diff_records([[1900, 1.0], [1901, 10.0]],
                                   [[1900, 2.0], [1901, 20.0]]))

In [ ]:
print('mis-matched years:', diff_records([[1900, 1.0]],
                                         [[1999, 2.0]]))

In [ ]:
print('left is shorter', diff_records([[1900, 1.0]],
                                      [[1900, 10.0], [1901, 20.0]]))

In [ ]:
print('right is shorter', diff_records([[1900, 1.0], [1901, 2.0]],
                                       [[1900, 10.0]]))

### 4. 시각화

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

australia = annual_mean_temp('AUS')
canada = annual_mean_temp('CAN')
diff = diff_records(australia, canada)
plt.plot(diff)
plt.show()

In [ ]:
import numpy as np
d = np.array(diff)
plt.plot(d[:, 0], d[:, 1])
plt.show()

In [ ]:
plt.plot(d[:, 0], d[:, 1])
plt.ylim((0,30))
plt.xlim((1950,2000))
plt.show()

### 5. 데이터 게시

In [ ]:
import csv

def save_records(filename, records):
    '''Save a list of [year, temp] pairs as CSV.'''
    with open(filename, 'w') as raw:
        writer = csv.writer(raw)
        writer.writerows(records)
        
# save_records('temp.csv', [[1, 2], [3, 4]])        #

In [ ]:
import csv

def save_records(left, right, records):
    '''Save a list of [year, temp] pairs as CSV.'''
    filename = left + '-' + right + '.csv'
    with open(filename, 'wb') as raw:
        writer = csv.writer(raw)
        writer.writerows(records)
        
australia = annual_mean_temp('KOR')
canada = annual_mean_temp('JPN')
diff = diff_records(australia, canada)        
save_records('KOR', 'JPN',  diff)

[Python3: writing csv files](http://stackoverflow.com/questions/7200606/python3-writing-csv-files)